In [1]:
%load_ext rpy2.ipython

In [18]:
%%R
library(mltools)
library(data.table)
library(dplyr)
library(ggplot2)
library(ggiraph)
library('mclust')

IN_DIR <- '/mnt/home/yuankeji/RanceLab/reticula_new/reticula/data/tcdd/output/'
OUT_DIR <- '/mnt/home/yuankeji/RanceLab/reticula_new/reticula/data/tcdd/validation/'

gnn_data <- read.csv(paste0(IN_DIR, 'predictions2.tsv'), sep = "\t", row.names = NULL)

In [7]:
%%R
str(gnn_data)

'data.frame':	383 obs. of  15 variables:
 $ project_ids       : chr  "SRP161461" "SRP161461" "SRP161461" "SRP161461" ...
 $ sample_ids        : chr  "SRR7817611" "SRR7817612" "SRR7817614" "SRR7817615" ...
 $ genders           : chr  "male" "male" "male" "male" ...
 $ doses             : num  0 0 0 0 0 30 30 30 30 30 ...
 $ target            : int  0 0 0 0 0 1 1 1 1 1 ...
 $ prediction        : int  0 0 0 0 0 1 1 1 1 1 ...
 $ confidence_class_0: num  0.812 0.865 0.911 0.826 0.765 ...
 $ confidence_class_1: num  0.1879 0.1345 0.0885 0.1742 0.2353 ...
 $ confidence_class_2: num  5.83e-06 1.16e-05 2.54e-06 3.88e-06 9.97e-06 ...
 $ confidence_class_3: num  7.35e-06 1.52e-05 3.44e-06 4.81e-06 1.39e-05 ...
 $ confidence_class_4: num  1.12e-05 2.34e-05 5.26e-06 6.87e-06 1.78e-05 ...
 $ confidence_class_5: num  6.03e-06 1.21e-05 3.16e-06 5.05e-06 1.40e-05 ...
 $ confidence_class_6: num  5.03e-06 1.09e-05 2.24e-06 3.33e-06 9.95e-06 ...
 $ confidence_class_7: num  5.18e-06 1.04e-05 2.67e-06 4.17e

In [19]:
%%R
gnn_ari <- adjustedRandIndex(gnn_data$target, gnn_data$prediction)
print(paste0("gnn_ari: ", gnn_ari))

[1] "gnn_ari: 0.434441205432818"


In [21]:
%%R
unique_dose <- unique(c(gnn_data$target, gnn_data$prediction))
names <- unique(gnn_data$dose)
gnn_results <- data.frame(names = names, dose = unique_dose)

gnn_results$TP <- 0
gnn_results$TN <- 0
gnn_results$FP <- 0
gnn_results$FN <- 0

for (dose in unique_dose) {
  for (i in 1:nrow(gnn_data)) {
    if (gnn_data$target[i] == dose) {
      if (gnn_data$prediction[i] == dose) {
          
        gnn_results[gnn_results$dose == dose, "TP"] <- gnn_results[gnn_results$dose == dose, "TP"] + 1
      } else {
        gnn_results[gnn_results$dose == dose, "FN"] <- gnn_results[gnn_results$dose == dose, "FN"] + 1
      }
    } else {
      if (gnn_data$prediction[i] == dose) {
        gnn_results[gnn_results$dose == dose, "FP"] <- gnn_results[gnn_results$dose == dose, "FP"] + 1
      } else {
        gnn_results[gnn_results$dose == dose, "TN"] <- gnn_results[gnn_results$dose == dose, "TN"] + 1
      }
    }
  }
}

gnn_results$POSITIVES <- gnn_results$TP + gnn_results$FN

gnn_results$MCC <- (gnn_results$TP * gnn_results$TN - gnn_results$FP * gnn_results$FN) / 
  sqrt((gnn_results$TP + gnn_results$FP) * (gnn_results$TP + gnn_results$FN) * (gnn_results$TN + gnn_results$FP) * (gnn_results$TN + gnn_results$FN))

gnn_results$TPR <- gnn_results$TP / (gnn_results$TP + gnn_results$FN)

gnn_results$FPR <- gnn_results$FP / (gnn_results$FP + gnn_results$TN)

gnn_results$TNR <- gnn_results$TN / (gnn_results$TN + gnn_results$FP)

gnn_results$FNR <- gnn_results$FN / (gnn_results$FN + gnn_results$TP)

gnn_results$Precision <- gnn_results$TP / (gnn_results$TP + gnn_results$FP)

gnn_results$Recall <- gnn_results$TP / (gnn_results$TP + gnn_results$FN)

gnn_results$F1_score <- 2 * (gnn_results$Precision * gnn_results$Recall)/(gnn_results$Precision + gnn_results$Recall)

print(gnn_results)

write.table(gnn_results, file = paste(OUT_DIR, "gnn_MCC_Result.txt"), sep="\t", row.names = FALSE, quote = FALSE)

  names dose TP  TN FP FN POSITIVES       MCC        TPR         FPR       TNR
1  0.00    0 92 211 62 18       110 0.5622124 0.83636364 0.227106227 0.7728938
2 30.00    1 91 254 19 19       110 0.7576757 0.82727273 0.069597070 0.9304029
3  0.01    2  2 358  2 21        23 0.1902371 0.08695652 0.005555556 0.9944444
4  0.03    3  5 355  5 18        23 0.3032008 0.21739130 0.013888889 0.9861111
5  0.30    4  3 355  5 20        23 0.1936206 0.13043478 0.013888889 0.9861111
6  1.00    5 16 344 13 10        26 0.5505240 0.61538462 0.036414566 0.9635854
7  3.00    6 13 346 13 11        24 0.4869838 0.54166667 0.036211699 0.9637883
8  0.10    7  7 355  6 15        22 0.3874967 0.31818182 0.016620499 0.9833795
9 10.00    8 20 352  9  2        22 0.7776698 0.90909091 0.024930748 0.9750693
         FNR Precision     Recall  F1_score
1 0.16363636 0.5974026 0.83636364 0.6969697
2 0.17272727 0.8272727 0.82727273 0.8272727
3 0.91304348 0.5000000 0.08695652 0.1481481
4 0.78260870 0.5000000 0.21739130 